# Izvrednjavanje funkcija

Računalo može izvoditi samo četiri osnovne operacije, `+`, `-`, `*` i `/` pa se sve ostale funkcije 
računaju pomoću polinoma (npr. Taylorova formula uz ocjenu ostatka ili bolje formule).

Neka je zadan polinom _stupnja_ $n$:

$$
p_n(x)=a_0+a_1 x+a_2x^2+a_3 x^3+\cdots + a_{n-1}x^{n-1}+a_n x^n,\quad a_n\neq 0.
$$

## Brzina

Direktno računanje vrijednosti $p_n(x)$ treba $O(n^2)$ operacija.

Uz __pamćenje potencija__ imamo sljedeći algoritam:

In [1]:
using Polynomials

In [2]:
function mypolyval(p::Poly,x::Number)
    s=p[0]
    t=one(typeof(p[0]))
    for i=1:length(p)-1
        t*=x
        s+=p[i]*t
    end
    s
    end 

mypolyval (generic function with 1 method)

In [3]:
p=Poly([1,2,3,4,5])

Poly(1 + 2*x + 3*x^2 + 4*x^3 + 5*x^4)

In [4]:
mypolyval(p,3)

547

In [5]:
mypolyval(p,π)

647.962560401659

Funkcija `mypolyval()` koristi $2n$ množenje i $n$ zbrajanja.

In [6]:
pbig=Poly(rand(1000));

In [8]:
@time mypolyval(pbig,1.5)

  0.000011 seconds (5 allocations: 176 bytes)


1.4486799670176103e176

In [10]:
@time polyval(pbig,1.5)

  0.000010 seconds (5 allocations: 176 bytes)


1.4486799670176067e176

__Hornerova shema__ (Horner, 1819, Newton 1669) treba $n$ množenja i $n$ zbrajanja:

In [11]:
function myhorner(p::Poly,x::Number)
    s=p[end]
    for i=length(p)-2:-1:0
        # s*=x
        # s+=p[i]
        s=s*x+p[i]
    end
    s
end

myhorner (generic function with 1 method)

In [12]:
myhorner(p,3)

547

In [14]:
@time myhorner(pbig,1.5)

  0.000009 seconds (5 allocations: 176 bytes)


1.4486799670176067e176

Hornerova shema je __optimalna__ u smislu da je općenito za izvrednjavanje polinoma $p_n(x)$ potrebno barem $n$ množenja. 

(Mogući su, naravno, posebni slučajevi, kao $x^{100}$.) 

## Točnost

Neka je $\hat q$ vrijednost $p_n(x)$ izračunata u aritmetici s točnošću stroja $\varepsilon$. Tada vrijedi ocjena
(vidi [Accuracy and Stability of Numerical Algorithms, str. 105][Hig96]):

$$
\big|\, p_n(x)-\hat q\,\big| \leq \frac{2n\varepsilon}{1-2n\varepsilon} \sum_{i=0}^n |a_i||x|^i.
$$

[Hig96]: https://books.google.hr/books?id=5tv3HdF-0N8C&printsec=frontcover&hl=hr#v=onepage&q&f=false    "Nick Higham, 'Accuracy and Stability of Numerical Algorithms', SIAM, Philadelphia, 1996"

In [15]:
p=Poly([1,2,3,4,5])
myhorner(p,sqrt(2))

41.142135623730965

In [16]:
pb=Poly(map(BigInt,[1,2,3,4,5]))

Poly(1 + 2*x + 3*x^2 + 4*x^3 + 5*x^4)

In [17]:
myhorner(pb,sqrt(map(BigFloat,2)))

4.114213562373095048801688724209698078569671875376948073176679737990732478462071e+01

In [18]:
myhorner(p,sqrt(200000))

2.0035837177182715e11

In [19]:
myhorner(pb,sqrt(map(BigFloat,200000)))

2.003583717718271573513413463506664716901809931137851879604936332676245815123154e+11

In [20]:
r=[1,sqrt(2),3,4,5,6,sqrt(50)]
p=poly(r)

Poly(-3600.0000000000005 + 10074.701294725885*x - 10926.667524715478*x^2 + 5983.714713523981*x^3 - 1813.4835482706842*x^4 + 308.2203461105329*x^5 - 27.48528137423857*x^6 + 1.0*x^7)

In [21]:
pb=poly(map(BigFloat,r))

Poly(-3.600000000000000379131566326426658519951798233677472418044762658695390200591646e+03 + 1.007470129472588617359567977839658265635786507617888371518728718445601089115371e+04*x - 1.092666752471547708236282815742121598471939907533833099921843218238493022909097e+04*x^2 + 5.983714713523981214300865042833422625843619822133098010478145789559079048558488e+03*x^3 - 1.813483548270684199169749849270990311652746621526228579007918029208923371697892e+03*x^4 + 3.082203461105328434184026381162611967403808614536830831056798962741538616683101e+02*x^5 - 2.748528137423857065080312622740166261792182922363281250000000000000000000000000e+01*x^6 + 1.000000000000000000000000000000000000000000000000000000000000000000000000000000*x^7)

In [22]:
myhorner(p,sqrt(2)+0.1)

-16.501829900900248

In [23]:
myhorner(pb,sqrt(map(BigFloat,2))+0.1)

-1.650182990089441570965221108407569790203119747704640347484765272697823211215861e+01

In [24]:
myhorner(p,-sqrt(10000))

-1.307549271826299e14

In [25]:
myhorner(pb,-sqrt(map(BigFloat,10000)))

-1.307549271826298681134778698255153254173615274698734585218439050325472949865002e+14